In [ ]:
# WEIGHTED EIGENSOUND: PARAMS + HELPERS

library(SoundShape)
library(geomorph)

OUT_ROOT <- "../results/asr_KNN/SYNTH"

#  distancias (salida de script Python)
MANIFEST_CSV <- "../results/asr_KNN/NN_AUDIO/ORIG/ASR/neighbors_manifest_ORIG.csv"

#  Parámetros eigensound
TLIM <- c(0, 5)
FLIM <- c(0, 10)    # kHz
WL   <- 512
OVLP <- 75
DBLEVEL_EIG <- 25
X_LENGTH <- 140
Y_LENGTH <- 94
LOG_SCALE <- FALSE

# Top-K
K_USE <- 16

#  Selección de nodos
MODE <- "all"   # "one" | "some" | "all"
NODE_ONE <- "47"
NODE_SOME <- c("47","52","70","88")

TAU_MODE <- "global"      # "global" | "per_node"
TAU      <- 0.2136408     # = median(dK) entre nodos (viene de arriba)


kernel_w <- function(d, tau) exp(- (d / tau)^2 )

n_eff <- function(w) (sum(w)^2) / sum(w^2)

wtd_quantile <- function(x, w, probs = c(0.025, 0.975)) {
  o <- order(x)
  x <- x[o]; w <- w[o]
  cw <- cumsum(w) / sum(w)
  sapply(probs, function(p) x[which(cw >= p)[1]])
}

# weighted mean shape (como mshape pero con pesos)
w_mshape <- function(A, w) {
  # A: p x k x n
  apply(A, c(1,2), function(v) sum(v * w) / sum(w))
}

# --- Helper: PCA ponderado (con parche de consistencia de signo en PC1) ---
wpca <- function(X, w, n_pc = 10) {
  w <- as.numeric(w)
  w_norm <- w / sum(w)

  mu <- as.numeric(colSums(X * w_norm))
  Xc <- sweep(X, 2, mu, "-")

  # aplica pesos por fila (observación)
  Xw <- Xc * sqrt(w_norm)

  r <- min(n_pc, nrow(X) - 1, ncol(X))
  sv <- svd(Xw, nu = 0, nv = r)

  rotation <- sv$v          # m x r
  values   <- sv$d^2        # eigenvalues (var por PC)
  totalvar <- sum(Xw^2)     # var total ponderada

  scores <- Xc %*% rotation # n x r

  if (ncol(scores) >= 1) {
    totE <- rowSums(X)  # energía total por vecino (antes de centrar)

    mx <- sum(w_norm * scores[, 1])
    my <- sum(w_norm * totE)

    cov_xy <- sum(w_norm * (scores[, 1] - mx) * (totE - my))
    var_x  <- sum(w_norm * (scores[, 1] - mx)^2)
    var_y  <- sum(w_norm * (totE - my)^2)

    cor_w <- cov_xy / sqrt(var_x * var_y)

    if (is.finite(cor_w) && cor_w < 0) {
      scores[, 1]   <- -scores[, 1]
      rotation[, 1] <- -rotation[, 1]
    }
  }

  list(mu = mu, rotation = rotation, values = values, totalvar = totalvar, scores = scores)
}

In [ ]:
# DIAGNÓSTICO DISTANCIAS + tau + n_eff

library(dplyr)
library(readr)

#  Resolver nodos a procesar 
if (MODE == "all") {
  node.dirs <- list.dirs(OUT_ROOT, recursive = FALSE, full.names = TRUE)
  node.dirs <- node.dirs[grepl("^node_", basename(node.dirs))]
  nodes <- gsub("^node_", "", basename(node.dirs))
} else if (MODE == "some") {
  nodes <- NODE_SOME
} else {
  nodes <- NODE_ONE
}

nodes
length(nodes)

#  Leer manifest 
man <- read_csv(MANIFEST_CSV, show_col_types = FALSE)
man$node <- as.character(man$node)

sel <- man %>% filter(node %in% nodes)

#  Resumen distancias top-100 por nodo 
dist_summary <- sel %>%
  group_by(node) %>%
  arrange(rank, .by_group = TRUE) %>%
  summarise(
    n    = n(),
    d1   = first(distance),
    dK   = distance[rank == K_USE][1],
    dmin = min(distance),
    q10  = quantile(distance, 0.10, names = FALSE),
    med  = median(distance),
    mean = mean(distance),
    q90  = quantile(distance, 0.90, names = FALSE),
    dmax = max(distance),
    .groups = "drop"
  ) %>%
  arrange(med)

cat("\n--- dist_summary (por nodo) ---\n")
print(dist_summary, n = Inf)

#  Tau global candidato (basado en dK mediano) 
TAU_GLOBAL_rankK  <- median(dist_summary$dK)
TAU_GLOBAL_median <- median(dist_summary$med)

cat("\n--- TAU candidates ---\n")
cat("TAU_GLOBAL (median of dK across nodes):", TAU_GLOBAL_rankK, "\n")
cat("TAU_GLOBAL (median of node medians):  ", TAU_GLOBAL_median, "\n")

cat("\n--- Heterogeneidad (distancias) ---\n")
cat("Range node medians:", range(dist_summary$med), "\n")
cat("Range dK:",         range(dist_summary$dK),  "\n")
cat("Ratio max/min (node median):", max(dist_summary$med) / min(dist_summary$med), "\n")
cat("Ratio max/min (dK):",         max(dist_summary$dK)  / min(dist_summary$dK),  "\n")

#  n_eff usando SOLO rank<=K_USE 
sel_k <- sel %>%
  filter(rank <= K_USE) %>%
  left_join(dist_summary %>% select(node, tau_per_node = dK), by = "node") %>%
  mutate(
    w_per_node = kernel_w(distance, tau_per_node),
    w_global   = kernel_w(distance, TAU_GLOBAL_rankK)
  )

neff_tbl <- sel_k %>%
  group_by(node) %>%
  summarise(
    tau_per_node  = first(tau_per_node),
    neff_per_node = n_eff(w_per_node),
    neff_global   = n_eff(w_global),
    .groups = "drop"
  ) %>%
  arrange(neff_per_node)

cat("\n--- n_eff (rank<=K_USE) ---\n")
print(neff_tbl, n = Inf)

cat("\nRange neff_per_node:", range(neff_tbl$neff_per_node), "\n")
cat("Range neff_global:  ", range(neff_tbl$neff_global), "\n")

#  print 
par(mar = c(9, 4, 2, 1))
boxplot(distance ~ node, data = sel, las = 2, outline = FALSE, cex.axis = 0.6,
        ylab = "KNN distance (cosine)",
        main = "Distribución de distancias (top-100) por nodo")
abline(h = TAU_GLOBAL_rankK, lty = 2)


In [ ]:
# EIGENSOUND PONDERADO + EXPORT

CSV_SUFFIX <- "_wtd"  # <- cambiar: "_wtd" | "_weighted" | "_distW" etc.

for (node in nodes) {

  ali.full <- file.path(OUT_ROOT, paste0("node_", node), "wav.at", "Aligned")
  store.at <- file.path(OUT_ROOT, paste0("node_", node), "store.at")
  dir.create(store.at, recursive = TRUE, showWarnings = FALSE)

  # carpeta temporal con top-K
  ali.tmp <- file.path(OUT_ROOT, paste0("node_", node), "_tmp_aligned_topK")
  unlink(ali.tmp, recursive = TRUE)
  dir.create(ali.tmp, recursive = TRUE, showWarnings = FALSE)

  wavs <- sort(list.files(ali.full, pattern="\\.wav$", full.names=TRUE))
  wavs_k <- wavs[1:min(K_USE, length(wavs))]
  file.copy(wavs_k, ali.tmp, overwrite = TRUE)

  #  distancias di desde filename 
  wav_names <- basename(wavs_k)
  d <- as.numeric(sub(".*_d([0-9.]+)_.*", "\\1", wav_names))

  tau_node <- d[min(K_USE, length(d))]
  tau_use <- if (TAU_MODE == "per_node") tau_node else TAU

  w <- kernel_w(d, tau_use)
  neff <- n_eff(w)

  cat("\n=== Node", node, "| K_USE =", length(wavs_k),
      "| tau =", round(tau_use, 6),
      "| n_eff =", round(neff, 3), "===\n")
  flush.console()

  res <- try({

    EIG3D <- SoundShape::eigensound(
      analysis.type = "threeDshape",
      wav.at        = ali.tmp,
      store.at      = store.at,
      flim          = FLIM,
      tlim          = TLIM,
      wl            = WL,
      ovlp          = OVLP,
      dBlevel       = DBLEVEL_EIG,
      x.length      = X_LENGTH,
      y.length      = Y_LENGTH,
      log.scale     = LOG_SCALE,
      plot.exp      = FALSE
    )

    p <- dim(EIG3D)[1]
    k <- dim(EIG3D)[2]

    #  CONSENSO ponderado 
    ref_w <- w_mshape(EIG3D, w)
    Mmean <- matrix(ref_w[,3], nrow = Y_LENGTH, ncol = X_LENGTH, byrow = TRUE)
    write.csv(Mmean, file.path(store.at, paste0("Mmean", CSV_SUFFIX, ".csv")), row.names = FALSE)

    #  PCA ponderado 
    Xmat <- geomorph::two.d.array(EIG3D)
    PCAw <- wpca(Xmat, w)

    pve <- PCAw$values / PCAw$totalvar

    pve12 <- data.frame(
      PC = c("PC1","PC2"),
      pve = c(pve[1], pve[2]),
      cum = c(pve[1], pve[1] + pve[2])
    )
    write.csv(pve12, file.path(store.at, paste0("PVE_PC1_PC2", CSV_SUFFIX, ".csv")), row.names = FALSE)

    #  cuantiles ponderados de scores (PC1/PC2) 
    s1 <- PCAw$scores[,1]
    s2 <- PCAw$scores[,2]

    q1 <- wtd_quantile(s1, w, probs = c(0.025, 0.975))
    q2 <- wtd_quantile(s2, w, probs = c(0.025, 0.975))

    v_PC1min <- PCAw$mu + q1[1] * PCAw$rotation[,1]
    v_PC1max <- PCAw$mu + q1[2] * PCAw$rotation[,1]
    v_PC2min <- PCAw$mu + q2[1] * PCAw$rotation[,2]
    v_PC2max <- PCAw$mu + q2[2] * PCAw$rotation[,2]

    shapes_mat <- rbind(
      PC1min = v_PC1min,
      PC1max = v_PC1max,
      PC2min = v_PC2min,
      PC2max = v_PC2max
    )

    shapes_arr <- geomorph::arrayspecs(shapes_mat, p, k)

    PC1min <- matrix(shapes_arr[,3,1], nrow = Y_LENGTH, ncol = X_LENGTH, byrow = TRUE)
    PC1max <- matrix(shapes_arr[,3,2], nrow = Y_LENGTH, ncol = X_LENGTH, byrow = TRUE)
    PC2min <- matrix(shapes_arr[,3,3], nrow = Y_LENGTH, ncol = X_LENGTH, byrow = TRUE)
    PC2max <- matrix(shapes_arr[,3,4], nrow = Y_LENGTH, ncol = X_LENGTH, byrow = TRUE)

    write.csv(PC1min, file.path(store.at, paste0("PC1min", CSV_SUFFIX, ".csv")), row.names = FALSE)
    write.csv(PC1max, file.path(store.at, paste0("PC1max", CSV_SUFFIX, ".csv")), row.names = FALSE)
    write.csv(PC2min, file.path(store.at, paste0("PC2min", CSV_SUFFIX, ".csv")), row.names = FALSE)
    write.csv(PC2max, file.path(store.at, paste0("PC2max", CSV_SUFFIX, ".csv")), row.names = FALSE)

    #  resumen de pesos 
    wsum <- data.frame(
      node = node,
      K_USE = length(wavs_k),
      TAU_MODE = TAU_MODE,
      tau = tau_use,
      n_eff = neff,
      d1 = d[1],
      dK = d[min(K_USE, length(d))],
      d_median = median(d)
    )
    write.csv(wsum, file.path(store.at, paste0("WEIGHTS_summary", CSV_SUFFIX, ".csv")), row.names = FALSE)

    cat("saved weighted CSVs with suffix:", CSV_SUFFIX, "\n")
    flush.console()

  }, silent = TRUE)

  unlink(ali.tmp, recursive = TRUE)

  if (inherits(res, "try-error")) {
    cat("FAIL node", node, "\n")
    cat(as.character(res), "\n")
    flush.console()
  }
}

In [ ]:

# CHECK: PC1 domina? (imprime resumen)

OUT_ROOT <- "../results/asr_KNN/SYNTH"

node.dirs <- list.dirs(OUT_ROOT, recursive = FALSE, full.names = TRUE)
node.dirs <- node.dirs[grepl("^node_", basename(node.dirs))]
nodes <- gsub("^node_", "", basename(node.dirs))

res <- data.frame(
  node = character(0),
  pve1 = numeric(0),
  pve2 = numeric(0),
  cum2 = numeric(0),
  stringsAsFactors = FALSE
)

for (node in nodes) {
  store.at <- file.path(OUT_ROOT, paste0("node_", node), "store.at")
  f <- file.path(store.at, "PVE_PC1_PC2_wtd.csv")

  if (file.exists(f)) {
    p <- read.csv(f, check.names = FALSE)

    p1 <- p$pve[p$PC == "PC1"][1]
    p2 <- p$pve[p$PC == "PC2"][1]
    c2 <- p$cum[p$PC == "PC2"][1]

    res <- rbind(res, data.frame(node=node, pve1=p1, pve2=p2, cum2=c2))
  }
}

if (nrow(res) == 0) {
  cat("No encontré PVE_PC1_PC2.csv en ningún nodo.\n")
} else {
  res <- res[order(-res$pve1), ]

  cat("Nodos con PVE guardado:", nrow(res), "/", length(nodes), "\n")
  cat("PC1 pve (min/med/max):",
      signif(min(res$pve1, na.rm=TRUE), 3), "/",
      signif(median(res$pve1, na.rm=TRUE), 3), "/",
      signif(max(res$pve1, na.rm=TRUE), 3), "\n")

  cat("PC2 pve (min/med/max):",
      signif(min(res$pve2, na.rm=TRUE), 3), "/",
      signif(median(res$pve2, na.rm=TRUE), 3), "/",
      signif(max(res$pve2, na.rm=TRUE), 3), "\n")

  bad <- res[which(res$pve2 >= res$pve1), ]
  cat("\nNodos donde PC2 >= PC1:", nrow(bad), "\n")
  if (nrow(bad) > 0) print(bad)

  cat("\nTop 10 nodos por PVE(PC1):\n")
  print(head(res, 10))
}

In [ ]:
# HEATMAP ΔPC1 -> store.at/heat_dPC1.png

plot_delta_div <- function(mat, out_png,
                           tlim = c(0,5), flim = c(0,10),
                           width = 2600, height = 1800, res = 300,
                           q_clip = 0.99, ncol = 201) {

  M <- as.numeric(quantile(abs(mat), probs = q_clip, na.rm = TRUE))
  if (!is.finite(M) || M == 0) M <- max(abs(mat), na.rm = TRUE)
  zlim <- c(-M, M)

  cols <- colorRampPalette(c("blue", "white", "red"))(ncol)

  x <- seq(tlim[1], tlim[2], length.out = ncol(mat))
  y <- seq(flim[1], flim[2], length.out = nrow(mat))

  png(out_png, width = width, height = height, res = res)

  layout(matrix(c(1,2), 1, 2), widths = c(4.6, 1.2))

  # panel principal (sin título)
  par(mar = c(5, 5, 2, 1) + 0.1)
  image(x, y, t(mat),
        col = cols, zlim = zlim,
        xlab = "Time (s)", ylab = "Frequency (kHz)",
        main = "")
  box()

  # colorbar
  par(mar = c(5, 1, 2, 4) + 0.1)
  zseq <- seq(zlim[1], zlim[2], length.out = ncol)
  image(x = 1, y = zseq, z = matrix(zseq, nrow = 1),
        col = cols, xlab = "", ylab = "", axes = FALSE)
  axis(4)
  mtext("ΔPC1", side = 4, line = 2.5)

  dev.off()
}

read_mat <- function(path) {
  m <- as.matrix(read.csv(path, check.names = FALSE))
  storage.mode(m) <- "numeric"
  m
}

for (node in nodes) {

  store.at <- file.path(OUT_ROOT, paste0("node_", node), "store.at")

  f_pc1min <- file.path(store.at, "PC1min_wtd.csv")
  f_pc1max <- file.path(store.at, "PC1max_wtd.csv")

  cat("\n--- Node", node, "---\n"); flush.console()

  if (file.exists(f_pc1min) && file.exists(f_pc1max)) {
    dPC1 <- read_mat(f_pc1max) - read_mat(f_pc1min)

    plot_delta_div(
      dPC1,
      out_png = file.path(store.at, "heat_dPC1_wtd.png"),
      tlim = TLIM, flim = FLIM
    )

    cat("saved heat_dPC1_wtd.png\n"); flush.console()
  } else {
    cat("missing PC1min/PC1max\n"); flush.console()
  }
}

In [ ]:
#  FIGURA GRID 9×5 ΔPC1 (PC1max - PC1min)


# PARÁMETROS (SOLO TAMAÑOS)
TICK_LABEL_CEX <- 2.25  # números de ticks (paneles)
X_AXIS_TITLE_CEX <- 3.50  # "Time (s)"
Y_AXIS_TITLE_CEX <- 2.40  # "Frequency (kHz)"
NODE_ID_CEX    <- 2.70  # ID de nodo
CBAR_AXIS_CEX  <- 2.20  # números de ticks (colorbar)
CBAR_LABEL_CEX <- 2.10  # etiqueta colorbar

# RUTAS / PNG
OUT_ROOT <- "../results/asr_KNN/SYNTH"
OUT_PNG  <- file.path(OUT_ROOT, "ALL_nodes_heat_dPC1_wtd.png")

png_width_px  <- 10000
png_height_px <- 7200
png_res_dpi   <- 300

# GRILLA / RANGOS
n_rows <- 9
n_cols <- 5

TLIM <- c(0, 5)    # s
FLIM <- c(0, 10)   # kHz
fmax_khz <- 8      # tope a mostrar (kHz)
FLIM_USE <- c(FLIM[1], min(FLIM[2], fmax_khz))

# ESTÉTICA
TICK_LEN <- -0.04
Y_LAS    <- 1

oma <- c(1.2, 9.2, 1.0, 1.6) 

# mar por panel
mar_def <- c(1.15, 1.35, 0.85, 0.85) 
mar_left_firstcol  <- 3.85
mar_bottom_lastrow <- 2.85

# filas extra
TIME_ROW_H <- 0.28
CBAR_ROW_H <- 0.55

# colorbar dentro de la última fila
cbar_width_frac <- 0.45
cbar_left_frac  <- 0.275

# mar interno de la región colorbar
cbar_mar <- c(4.2, 2.2, 0.4, 2.2) 
cbar_label_line <- 3.4
cbar_label <- "ΔPC1"

# label Y global
y_label_outer_line <- 3.6

# colormap / zlim robusto
cmap_n    <- 201
cmap_cols <- c("blue", "white", "red")
z_quant   <- 0.99

# HELPERS
read_mat <- function(path) {
  m <- as.matrix(read.csv(path, check.names = FALSE))
  storage.mode(m) <- "numeric"
  m
}

label_box_topright <- function(xlim, ylim, label, cex, pad_x = 0.02, pad_y = 0.02) {
  x0 <- xlim[2] - pad_x * diff(xlim)
  y0 <- ylim[2] - pad_y * diff(ylim)
  w  <- strwidth(label,  cex = cex, units = "user")
  h  <- strheight(label, cex = cex, units = "user")
  rect(x0 - w - 0.40*w, y0 - h - 0.40*h, x0 + 0.40*w, y0 + 0.40*h, col = "white", border = NA)
  text(x0, y0, label, adj = c(1, 1), cex = cex, col = "black")
}

# CARGA ΔPC1 (solo nodos válidos)
node.dirs <- list.dirs(OUT_ROOT, recursive = FALSE, full.names = TRUE)
node.dirs <- node.dirs[grepl("^node_", basename(node.dirs))]
node.ids  <- gsub("^node_", "", basename(node.dirs))

if (all(grepl("^[0-9]+$", node.ids))) node.ids <- node.ids[order(as.integer(node.ids))] else node.ids <- sort(node.ids)

mat_list <- list()
for (node in node.ids) {
  store.at <- file.path(OUT_ROOT, paste0("node_", node), "store.at")
  f_min <- file.path(store.at, "PC1min_wtd.csv")
  f_max <- file.path(store.at, "PC1max_wtd.csv")
  if (!file.exists(f_min) || !file.exists(f_max)) next

  dm <- read_mat(f_max) - read_mat(f_min)

  mat_list[[node]] <- dm
}
stopifnot(length(mat_list) > 0)

# colormap (fijo; la escala se calculará POR NODO)
COLS <- colorRampPalette(cmap_cols)(cmap_n)

# nodos a plotear (hasta 45)
n_panels <- n_rows * n_cols
nodes_use <- names(mat_list)
if (all(grepl("^[0-9]+$", nodes_use))) nodes_use <- nodes_use[order(as.integer(nodes_use))] else nodes_use <- sort(nodes_use)

# LAYOUT (paneles + Time row + Colorbar)
plot_ids <- matrix(1:n_panels, nrow = n_rows, ncol = n_cols, byrow = TRUE)
time_id <- n_panels + 1
cbar_id <- n_panels + 2

lay <- rbind(
  plot_ids,
  rep(time_id, n_cols),
  rep(0L, n_cols)  # fila colorbar (solo algunas columnas)
)

n_cb_cols <- max(1, floor(n_cols * cbar_width_frac))
start_col <- max(1, floor(1 + cbar_left_frac * (n_cols - n_cb_cols)))
end_col   <- min(n_cols, start_col + n_cb_cols - 1)
lay[n_rows + 2, start_col:end_col] <- cbar_id

heights <- c(rep(1, n_rows), TIME_ROW_H, CBAR_ROW_H)

# RENDER
png(OUT_PNG, width = png_width_px, height = png_height_px, res = png_res_dpi)
layout(lay, heights = heights)
par(oma = oma)

#  45 paneles 
for (idx in 1:n_panels) {
  col_i <- ((idx - 1) %% n_cols) + 1
  row_i <- ceiling(idx / n_cols)

  mar <- mar_def
  if (col_i == 1)      mar[2] <- mar_left_firstcol
  if (row_i == n_rows) mar[1] <- mar_bottom_lastrow
  par(mar = mar)

  if (idx <= length(nodes_use)) {
    node <- nodes_use[idx]
    mat_full <- mat_list[[node]]


    y_full <- seq(FLIM[1], FLIM[2], length.out = nrow(mat_full))
    keep   <- which(y_full <= fmax_khz)  # mostrar solo 0–8 kHz
    mat    <- mat_full[keep, , drop = FALSE]

    x <- seq(TLIM[1], TLIM[2], length.out = ncol(mat_full))
    y <- y_full[keep]

    plot.new(); par(xaxs = "i", yaxs = "i"); plot.window(xlim = range(x), ylim = range(y))

M_node <- as.numeric(quantile(abs(mat_full), probs = z_quant, na.rm = TRUE))
if (!is.finite(M_node) || M_node == 0) M_node <- max(abs(mat_full), na.rm = TRUE)
zlim_node <- c(-M_node, M_node)

image(x, y, t(mat), col = COLS, zlim = zlim_node, axes = FALSE, xlab = "", ylab = "", add = TRUE)
    box()

    if (col_i == 1)       axis(2, at = 0:8, labels = 0:8, tck = TICK_LEN, cex.axis = TICK_LABEL_CEX, las = Y_LAS)
    if (row_i == n_rows)  axis(1, at = 0:5, labels = 0:5, tck = TICK_LEN, cex.axis = TICK_LABEL_CEX)

    label_box_topright(range(x), range(y), node, cex = NODE_ID_CEX, pad_y = 0.05)
  } else {
    plot.new(); box()
  }
}

#  fila "Time (s)" 
par(mar = c(0, 0, 0, 0))
plot.new()
text(0.5, 0.5, "Time (s)", cex = X_AXIS_TITLE_CEX)

#  colorbar 
par(mar = c(cbar_mar[1], cbar_mar[2], cbar_mar[3], cbar_mar[4]))
plot.new()

zseq <- seq(-1, 1, length.out = cmap_n)
par(xaxs = "i", yaxs = "i"); plot.window(xlim = range(zseq), ylim = c(0, 1))
image(x = zseq, y = 0.82, z = matrix(zseq, nrow = length(zseq), ncol = 1), col = COLS, add = TRUE)
axis(1, at = c(-1, 1), labels = c("PC1min", "PC1max"), line = 0, cex.axis = CBAR_AXIS_CEX, tck = TICK_LEN)
mtext(cbar_label, side = 1, line = cbar_label_line, cex = CBAR_LABEL_CEX)

#  label Y global 
mtext("Frequency (kHz)", side = 2, outer = TRUE, line = y_label_outer_line, cex = Y_AXIS_TITLE_CEX)

dev.off()
cat(sprintf("Saved: %s
", OUT_PNG))


In [ ]:
# CELDA — FIGURA GRID 9×5 ΔPC1 + overlay Mmean + ridge cuantil alto


# PARÁMETROS
# textos
TICK_LABEL_CEX <- 2.25
NODE_ID_CEX    <- 2.70
CBAR_AXIS_CEX  <- 2.20
CBAR_LABEL_CEX <- 2.10

X_AXIS_TITLE_CEX <- 3.50  # "Time (s)"
Y_AXIS_TITLE_CEX <- 2.40  # "Frequency (kHz)"

# recorte / ejes
TLIM <- c(0, 5)    # s
FLIM <- c(0, 10)   # kHz (escala original)
fmax_khz <- 8      # SOLO para visualización (0–8)

# ΔPC1 escala por nodo
DELTA_QCLIP <- 0.99

# overlay Mmean
MEAN_FILE   <- "Mmean_wtd.csv"  # dentro de store.at
MEAN_QCLIP  <- 0.99          # para mapear grises (por nodo)
MEAN_ALPHA  <- 0.25          # opacidad del overlay Mmean (0=transparente, 1=opaco)
MEAN_GAMMA  <- 0.10          # contraste no-lineal del overlay

# ridge (cuantil alto sobre Mmean)
RIDGE_Q         <- 0.90
RIDGE_SMOOTH_K  <- 9         # impar (suavizado; 0 para no suavizar)
RIDGE_USE_HALO  <- FALSE       # TRUE = con halo blanco; FALSE = solo línea negra
RIDGE_LWD        <- 3.6
RIDGE_HALO_LWD   <- 6.5        # grosor del halo (solo si RIDGE_USE_HALO=TRUE)
RIDGE_HALO_COL   <- "white"
RIDGE_COL        <- "black"

# colormap ΔPC1
cmap_n    <- 201
cmap_cols <- c("blue", "white", "red")

# RUTAS / PNG
OUT_ROOT <- "../results/asr_KNN/SYNTH"
OUT_PNG  <- file.path(OUT_ROOT, "ALL_nodes_heat_dPC1_mean_wtd.png")

png_width_px  <- 10000
png_height_px <- 8200
png_res_dpi   <- 300

# GRILLA / MÁRGENES
n_rows <- 9
n_cols <- 5
n_panels <- n_rows * n_cols

# separación entre paneles
mar_def <- c(1.15, 1.35, 0.85, 0.85)  # b, l, t, r
mar_left_firstcol  <- 3.85
mar_bottom_lastrow <- 2.85

# margen externo de toda la figura
oma <- c(1.2, 9.2, 1.0, 1.6)  # bottom, left, top, right

# fila extra para "Time (s)" y colorbar
TIME_ROW_H <- 0.28
CBAR_ROW_H <- 0.55

# colorbar (posición y espacio)
cbar_width_frac <- 0.45
cbar_left_frac  <- 0.275
cbar_mar <- c(4.2, 2.2, 0.4, 2.2)  # b, l, t, r
cbar_label_line <- 3.4
cbar_label <- "ΔPC1"

# ticks
TICK_LEN <- -0.04
Y_LAS    <- 1

# HELPERS
read_mat <- function(path) {
  m <- as.matrix(read.csv(path, check.names = FALSE))
  storage.mode(m) <- "numeric"
  m
}

label_box_topright <- function(xlim, ylim, label, cex, pad_x = 0.02, pad_y = 0.05) {
  x0 <- xlim[2] - pad_x * diff(xlim)
  y0 <- ylim[2] - pad_y * diff(ylim)
  w  <- strwidth(label,  cex = cex, units = "user")
  h  <- strheight(label, cex = cex, units = "user")
  rect(x0 - w - 0.40*w, y0 - h - 0.40*h, x0 + 0.40*w, y0 + 0.40*h, col = "white", border = NA)
  text(x0, y0, label, adj = c(1, 1), cex = cex, col = "black")
}

ridge_from_mean <- function(mean_mat, y_vec, q = 0.90, smooth_k = 9) {
  nT <- ncol(mean_mat)
  r  <- rep(NA_real_, nT)
  for (j in 1:nT) {
    v <- mean_mat[, j]
    # robusto: permite matrices con valores negativos o centrados
    v <- v - min(v, na.rm = TRUE)
    v <- pmax(v, 0)
    s <- sum(v, na.rm = TRUE)
    if (!is.finite(s) || s <= 0) next
    cs <- cumsum(v) / s
    idx <- which(cs >= q)[1]
    if (!is.na(idx)) r[j] <- y_vec[idx]
  }
  if (smooth_k && smooth_k >= 3 && (smooth_k %% 2 == 1)) {
    ok <- which(is.finite(r))
    if (length(ok) >= smooth_k) r[ok] <- stats::runmed(r[ok], k = smooth_k, endrule = "median")
  }
  r
}

# CARGA POR NODO (ΔPC1 + Mmean) EN ESCALA ORIGINAL 0–10
node.dirs <- list.dirs(OUT_ROOT, recursive = FALSE, full.names = TRUE)
node.dirs <- node.dirs[grepl("^node_", basename(node.dirs))]
node.ids  <- gsub("^node_", "", basename(node.dirs))

if (all(grepl("^[0-9]+$", node.ids))) node.ids <- node.ids[order(as.integer(node.ids))] else node.ids <- sort(node.ids)

mat_delta <- list()
mat_mean  <- list()

for (node in node.ids) {
  store.at <- file.path(OUT_ROOT, paste0("node_", node), "store.at")
  f_min <- file.path(store.at, "PC1min_wtd.csv")
  f_max <- file.path(store.at, "PC1max_wtd.csv")
  f_mean <- file.path(store.at, MEAN_FILE)

  if (!file.exists(f_min) || !file.exists(f_max)) next
  if (!file.exists(f_mean)) next

  d <- read_mat(f_max) - read_mat(f_min)
  m <- read_mat(f_mean)

  # deben calzar
  if (!all(dim(d) == dim(m))) next

  mat_delta[[node]] <- d
  mat_mean[[node]]  <- m
}

stopifnot(length(mat_delta) > 0)

nodes_use <- intersect(names(mat_delta), names(mat_mean))
if (all(grepl("^[0-9]+$", nodes_use))) nodes_use <- nodes_use[order(as.integer(nodes_use))] else nodes_use <- sort(nodes_use)

# colormaps
COLS_DELTA <- colorRampPalette(cmap_cols)(cmap_n)
COLS_MEAN  <- grDevices::adjustcolor(grDevices::colorRampPalette(c("white", "black"))(cmap_n), alpha.f = MEAN_ALPHA)

# LAYOUT
plot_ids <- matrix(1:n_panels, nrow = n_rows, ncol = n_cols, byrow = TRUE)
time_id <- n_panels + 1
cbar_id <- n_panels + 2

lay <- rbind(
  plot_ids,
  rep(time_id, n_cols),
  rep(0L, n_cols)
)

n_cb_cols <- max(1, floor(n_cols * cbar_width_frac))
start_col <- max(1, floor(1 + cbar_left_frac * (n_cols - n_cb_cols)))
end_col   <- min(n_cols, start_col + n_cb_cols - 1)
lay[n_rows + 2, start_col:end_col] <- cbar_id

heights <- c(rep(1, n_rows), TIME_ROW_H, CBAR_ROW_H)

# RENDER
png(OUT_PNG, width = png_width_px, height = png_height_px, res = png_res_dpi)
layout(lay, heights = heights)
par(oma = oma)

for (idx in 1:n_panels) {
  col_i <- ((idx - 1) %% n_cols) + 1
  row_i <- ceiling(idx / n_cols)

  mar <- mar_def
  if (col_i == 1)      mar[2] <- mar_left_firstcol
  if (row_i == n_rows) mar[1] <- mar_bottom_lastrow
  par(mar = mar)

  if (idx <= length(nodes_use)) {
    node <- nodes_use[idx]

    d_full <- mat_delta[[node]]
    m_full <- mat_mean[[node]]

    # ejes originales (0–10), pero mostramos solo <= fmax_khz
    y_full <- seq(FLIM[1], FLIM[2], length.out = nrow(d_full))
    keep   <- which(y_full <= fmax_khz)

    d <- d_full[keep, , drop = FALSE]
    m <- m_full[keep, , drop = FALSE]

    x <- seq(TLIM[1], TLIM[2], length.out = ncol(d_full))
    y <- y_full[keep]

    # zlim por nodo para ΔPC1 (igual que individuales)
    M_node <- as.numeric(quantile(abs(d_full), probs = DELTA_QCLIP, na.rm = TRUE))
    if (!is.finite(M_node) || M_node == 0) M_node <- max(abs(d_full), na.rm = TRUE)
    zlim_node <- c(-M_node, M_node)

    # zlim por nodo para Mmean (solo para mapear grises; robusto)
vals_m <- as.numeric(m_full)
vals_m <- vals_m[is.finite(vals_m)]
if (length(vals_m) == 0) {
  zlim_mean <- NULL
} else {
  lo_m <- as.numeric(stats::quantile(vals_m, probs = 0.01, na.rm = TRUE))
  hi_m <- as.numeric(stats::quantile(vals_m, probs = MEAN_QCLIP, na.rm = TRUE))
  if (!is.finite(lo_m)) lo_m <- min(vals_m)
  if (!is.finite(hi_m)) hi_m <- max(vals_m)
  if (!is.finite(lo_m) || !is.finite(hi_m)) {
    zlim_mean <- NULL
  } else {
    # asegura límites válidos
    if (hi_m <= lo_m) hi_m <- lo_m + 1e-6
    zlim_mean <- c(lo_m, hi_m)
  }
}

    # ridge (cuantil alto) sobre Mmean
    ridge_y <- ridge_from_mean(m, y, q = RIDGE_Q, smooth_k = RIDGE_SMOOTH_K)

    # --- dibuja ---
    plot.new(); par(xaxs = "i", yaxs = "i"); plot.window(xlim = range(x), ylim = range(y))

    # ΔPC1
    image(x, y, t(d), col = COLS_DELTA, zlim = zlim_node, axes = FALSE, xlab = "", ylab = "", add = TRUE)

    # Mmean (gris transparente) — normalizado + gamma para que se note
if (!is.null(zlim_mean)) {
  lo <- zlim_mean[1]; hi <- zlim_mean[2]
  mm <- (m - lo) / (hi - lo)
  # IMPORTANT: no usar pmax/pmin aquí porque pueden borrar las dimensiones de la matriz
  mm[!is.finite(mm)] <- 0
  mm[mm < 0] <- 0
  mm[mm > 1] <- 1
  mm <- mm^MEAN_GAMMA
  image(x, y, t(mm), col = COLS_MEAN, zlim = c(0, 1), axes = FALSE, xlab = "", ylab = "", add = TRUE)
}

    # ridge (opcionalmente con halo)
if (isTRUE(RIDGE_USE_HALO)) {
  lines(x, ridge_y, col = RIDGE_HALO_COL, lwd = RIDGE_HALO_LWD)
}
lines(x, ridge_y, col = RIDGE_COL, lwd = RIDGE_LWD)

    box()

    if (col_i == 1) {
      axis(2, at = 0:fmax_khz, labels = 0:fmax_khz, tck = TICK_LEN, cex.axis = TICK_LABEL_CEX, las = Y_LAS)
    }
    if (row_i == n_rows) {
      axis(1, at = 0:5, labels = 0:5, tck = TICK_LEN, cex.axis = TICK_LABEL_CEX)
    }

    label_box_topright(range(x), range(y), node, cex = NODE_ID_CEX)

  } else {
    plot.new(); box()
  }
}

# --- fila "Time (s)" ---
par(mar = c(0, 0, 0, 0))
plot.new()
text(0.5, 0.5, "Time (s)", cex = X_AXIS_TITLE_CEX)

# --- colorbar simbólica (escala por nodo) ---
par(mar = c(cbar_mar[1], cbar_mar[2], cbar_mar[3], cbar_mar[4]))
plot.new()

zseq <- seq(-1, 1, length.out = cmap_n)
par(xaxs = "i", yaxs = "i"); plot.window(xlim = range(zseq), ylim = c(0, 1))
image(x = zseq, y = 0.82, z = matrix(zseq, nrow = length(zseq), ncol = 1), col = COLS_DELTA, add = TRUE)
axis(1, at = c(-1, 1), labels = c("PC1min", "PC1max"), line = 0, cex.axis = CBAR_AXIS_CEX, tck = TICK_LEN)
mtext(cbar_label, side = 1, line = cbar_label_line, cex = CBAR_LABEL_CEX)

# --- label Y global ---
mtext("Frequency (kHz)", side = 2, outer = TRUE, line = 3.6, cex = Y_AXIS_TITLE_CEX)

dev.off()
cat(sprintf("Saved: %s
", OUT_PNG))


In [ ]:
# LIENZO

# PARÁMETROS
OUT_ROOT <- "../results/asr_KNN/SYNTH"

MEAN_FILE <- "Mmean_wtd.csv"
OUT_NAME  <- "heat_dPC1_mean_wtd.png"

TLIM <- c(0, 5)
FLIM <- c(0, 10)
fmax_khz <- 8

DELTA_QCLIP <- 0.99
MEAN_QCLIP  <- 0.99
MEAN_ALPHA  <- 0.25
MEAN_GAMMA  <- 1.80

RIDGE_Q        <- 0.90
RIDGE_SMOOTH_K <- 9
RIDGE_USE_HALO <- FALSE
RIDGE_LWD      <- 3.2
RIDGE_HALO_LWD <- 5.8
RIDGE_HALO_COL <- "white"
RIDGE_COL      <- "black"

cmap_n    <- 201
COLS_DELTA <- colorRampPalette(c("blue","white","red"))(cmap_n)
COLS_MEAN_OVERLAY <- grDevices::adjustcolor(colorRampPalette(c("white","black"))(cmap_n), alpha.f = MEAN_ALPHA)
COLS_MEAN_BAR <- grDevices::adjustcolor(
  colorRampPalette(c("white","black"))(cmap_n),
  alpha.f = MEAN_ALPHA
)


width  <- 2600
height <- 1800
res    <- 300

TICK_CEX <- 1.25
LAB_CEX  <- 1.35

BAR_TICK_CEX <- 1.10
BAR_LAB_CEX  <- 1.05

FIG_MAIN <- c(0.04, 0.99, 0.07, 0.99)

BAR_W_FRAC   <- 0.23
BAR_GAP_FRAC <- 0.035
BAR_Y0_FRAC  <- 0.10
BAR_Y1_FRAC  <- 0.60
BAR_MAR      <- c(1.10, 0.25, 0.55, 0.25)

# HELPERS
read_mat <- function(path) {
  m <- as.matrix(read.csv(path, check.names = FALSE))
  storage.mode(m) <- "numeric"
  m
}

ridge_from_mean <- function(mean_mat, y_vec, q = 0.90, smooth_k = 9) {
  nT <- ncol(mean_mat)
  r  <- rep(NA_real_, nT)
  for (j in 1:nT) {
    v <- mean_mat[, j]
    v <- v - min(v, na.rm = TRUE)
    v <- pmax(v, 0)
    s <- sum(v, na.rm = TRUE)
    if (!is.finite(s) || s <= 0) next
    cs <- cumsum(v) / s
    idx <- which(cs >= q)[1]
    if (!is.na(idx)) r[j] <- y_vec[idx]
  }
  if (smooth_k && smooth_k >= 3 && (smooth_k %% 2 == 1)) {
    ok <- which(is.finite(r))
    if (length(ok) >= smooth_k) r[ok] <- stats::runmed(r[ok], k = smooth_k, endrule = "median")
  }
  r
}

plot_node_panel <- function(d_full, m_full, out_png) {
  y_full <- seq(FLIM[1], FLIM[2], length.out = nrow(d_full))
  keep   <- which(y_full <= fmax_khz)

  d <- d_full[keep, , drop = FALSE]
  m <- m_full[keep, , drop = FALSE]

  x <- seq(TLIM[1], TLIM[2], length.out = ncol(d_full))
  y <- y_full[keep]

  # zlim ΔPC1 por nodo
  M_node <- as.numeric(stats::quantile(abs(d_full), probs = DELTA_QCLIP, na.rm = TRUE))
  if (!is.finite(M_node) || M_node == 0) M_node <- max(abs(d_full), na.rm = TRUE)
  zlim_delta <- c(-M_node, M_node)

  # zlim Mean por nodo
  vals_m <- as.numeric(m_full)
  vals_m <- vals_m[is.finite(vals_m)]
  if (length(vals_m) == 0) return(invisible(FALSE))

  lo_m <- as.numeric(stats::quantile(vals_m, probs = 0.10,       na.rm = TRUE))
  hi_m <- as.numeric(stats::quantile(vals_m, probs = MEAN_QCLIP, na.rm = TRUE))
  if (!is.finite(lo_m)) lo_m <- min(vals_m)
  if (!is.finite(hi_m)) hi_m <- max(vals_m)
  if (!is.finite(lo_m) || !is.finite(hi_m)) return(invisible(FALSE))
  if (hi_m <= lo_m) hi_m <- lo_m + 1e-6

  mm <- (m - lo_m) / (hi_m - lo_m)
  mm[!is.finite(mm)] <- 0
  mm[mm < 0] <- 0
  mm[mm > 1] <- 1
  mm <- mm^MEAN_GAMMA

  ridge_y <- ridge_from_mean(m, y, q = RIDGE_Q, smooth_k = RIDGE_SMOOTH_K)

  png(out_png, width = width, height = height, res = res)

  #  MAIN
  par(fig = FIG_MAIN, new = FALSE)
  par(mar = c(6.8, 5.2, 1.6, 1.0) + 0.1, xaxs = "i", yaxs = "i")
  plot.new(); plot.window(xlim = range(x), ylim = c(FLIM[1], fmax_khz))

  # ORDEN DE CAPAS
  # Base: Mmean (grises, con alpha)
  image(x, y, t(mm), col = COLS_MEAN_OVERLAY, zlim = c(0, 1),     axes = FALSE, xlab = "", ylab = "", add = TRUE)
  # Encima: ΔPC1 (rojo-azul)
  image(x, y, t(d),  col = COLS_DELTA,        zlim = zlim_delta,  axes = FALSE, xlab = "", ylab = "", add = TRUE)

  # Encima de todo: ridge
  if (isTRUE(RIDGE_USE_HALO)) lines(x, ridge_y, col = RIDGE_HALO_COL, lwd = RIDGE_HALO_LWD)
  lines(x, ridge_y, col = RIDGE_COL, lwd = RIDGE_LWD)

  box()
  axis(1, at = 0:5, labels = 0:5, cex.axis = TICK_CEX)
  axis(2, at = seq(0, fmax_khz, by = 1), labels = seq(0, fmax_khz, by = 1), las = 1, cex.axis = TICK_CEX)
  mtext("Time (s)", side = 1, line = 2.9, cex = LAB_CEX, adj = 1)
  mtext("Frequency (kHz)", side = 2, line = 3.3, cex = LAB_CEX, adj = 1)

  #  BARRAS
  plt <- par("plt")
  xL <- plt[1]; xR <- plt[2]
  fig_main <- par("fig")
  yB <- fig_main[3]
  yT <- plt[3]

  barY0 <- yB + BAR_Y0_FRAC * (yT - yB)
  barY1 <- yB + BAR_Y1_FRAC * (yT - yB)

  W <- (xR - xL)
  bw <- BAR_W_FRAC * W
  bg <- BAR_GAP_FRAC * W

  x0a <- xL
  x1a <- x0a + bw
  x0b <- x1a + bg
  x1b <- x0b + bw

  draw_bar <- function(fig, cols, labels_at, labels_txt, title) {
    par(fig = fig, new = TRUE)
    par(mar = BAR_MAR, xaxs = "i", yaxs = "i")
    par(xpd = NA)

    plot.new(); plot.window(xlim = TLIM, ylim = c(0, 1))

    xb <- seq(TLIM[1], TLIM[2], length.out = cmap_n)
    image(x = xb, y = 0.5,
          z = matrix(xb, nrow = length(xb), ncol = 1),
          col = cols, add = TRUE)

    axis(1, at = labels_at, labels = labels_txt, cex.axis = BAR_TICK_CEX)
    mtext(title, side = 3, line = 0.05, cex = BAR_LAB_CEX)
  }

  draw_bar(fig = c(x0a, x1a, barY0, barY1),
           cols = COLS_DELTA,
           labels_at = c(TLIM[1], mean(TLIM), TLIM[2]),
           labels_txt = c("min", "0", "max"),
           title = "ΔPC1")

  draw_bar(fig = c(x0b, x1b, barY0, barY1),
           cols = COLS_MEAN_BAR,
           labels_at = c(TLIM[1], TLIM[2]),
           labels_txt = c("min", "max"),
           title = "Mean")

  dev.off()
  invisible(TRUE)
}

# LOOP NODOS
node.dirs <- list.dirs(OUT_ROOT, recursive = FALSE, full.names = TRUE)
node.dirs <- node.dirs[grepl("^node_", basename(node.dirs))]
node.ids  <- gsub("^node_", "", basename(node.dirs))
if (all(grepl("^[0-9]+$", node.ids))) node.ids <- node.ids[order(as.integer(node.ids))] else node.ids <- sort(node.ids)

ok_n <- 0
for (node in node.ids) {
  store.at <- file.path(OUT_ROOT, paste0("node_", node), "store.at")
  f_min  <- file.path(store.at, "PC1min_wtd.csv")
  f_max  <- file.path(store.at, "PC1max_wtd.csv")
  f_mean <- file.path(store.at, MEAN_FILE)

  if (!file.exists(f_min) || !file.exists(f_max) || !file.exists(f_mean)) next

  d_full <- read_mat(f_max) - read_mat(f_min)
  m_full <- read_mat(f_mean)
  if (!all(dim(d_full) == dim(m_full))) next

  out_png <- file.path(store.at, OUT_NAME)
  if (isTRUE(plot_node_panel(d_full, m_full, out_png))) {
    ok_n <- ok_n + 1
    cat("saved:", out_png, "\n")
  }
}

cat("DONE. Panels saved:", ok_n, "\n")


In [ ]:
# LIENZO SEPARADO

OUT_CLEAN <- file.path(OUT_ROOT, "figuras_limpias")
dir.create(OUT_CLEAN, showWarnings = FALSE, recursive = TRUE)

OUT_NAME_CLEAN <- sub("\\.png$", "_clean.png", OUT_NAME)

plot_node_panel_clean <- function(d_full, m_full, out_png) {
  # ejes originales, recorte solo para mostrar
  y_full <- seq(FLIM[1], FLIM[2], length.out = nrow(d_full))
  keep   <- which(y_full <= fmax_khz)

  d <- d_full[keep, , drop = FALSE]
  m <- m_full[keep, , drop = FALSE]

  x <- seq(TLIM[1], TLIM[2], length.out = ncol(d_full))
  y <- y_full[keep]

  # zlim ΔPC1 por nodo
  M_node <- as.numeric(stats::quantile(abs(d_full), probs = DELTA_QCLIP, na.rm = TRUE))
  if (!is.finite(M_node) || M_node == 0) M_node <- max(abs(d_full), na.rm = TRUE)
  zlim_delta <- c(-M_node, M_node)

  # zlim Mean por nodo + gamma
  vals_m <- as.numeric(m_full)
  vals_m <- vals_m[is.finite(vals_m)]
  if (length(vals_m) == 0) return(invisible(FALSE))

  lo_m <- as.numeric(stats::quantile(vals_m, probs = 0.10,       na.rm = TRUE))
  hi_m <- as.numeric(stats::quantile(vals_m, probs = MEAN_QCLIP, na.rm = TRUE))
  if (!is.finite(lo_m)) lo_m <- min(vals_m)
  if (!is.finite(hi_m)) hi_m <- max(vals_m)
  if (hi_m <= lo_m) hi_m <- lo_m + 1e-6

  mm <- (m - lo_m) / (hi_m - lo_m)
  mm[!is.finite(mm)] <- 0
  mm[mm < 0] <- 0
  mm[mm > 1] <- 1
  mm <- mm^MEAN_GAMMA

  # ridge
  ridge_y <- ridge_from_mean(m, y, q = RIDGE_Q, smooth_k = RIDGE_SMOOTH_K)

  png(out_png, width = width, height = height, res = res)

  # ocupar TODO el device, sin márgenes ni marco
  par(mar = c(0, 0, 0, 0), oma = c(0, 0, 0, 0), xaxs = "i", yaxs = "i")
  par(fig = c(0, 1, 0, 1), new = FALSE)

  plot.new()
  plot.window(xlim = range(x), ylim = c(FLIM[1], fmax_khz), xaxs = "i", yaxs = "i")

  # ORDEN DE CAPAS
  # Base: Mmean (gris, alpha)
  image(x, y, t(mm), col = COLS_MEAN_OVERLAY, zlim = c(0, 1),     axes = FALSE, xlab = "", ylab = "", add = TRUE)
  # Encima: ΔPC1 (azul-blanco-rojo)
  image(x, y, t(d),  col = COLS_DELTA,        zlim = zlim_delta,  axes = FALSE, xlab = "", ylab = "", add = TRUE)

  # Encima de todo: ridge
  if (isTRUE(RIDGE_USE_HALO)) lines(x, ridge_y, col = RIDGE_HALO_COL, lwd = RIDGE_HALO_LWD)
  lines(x, ridge_y, col = RIDGE_COL, lwd = RIDGE_LWD)

  dev.off()
  invisible(TRUE)
}

ok_n_clean <- 0
for (node in node.ids) {
  store.at <- file.path(OUT_ROOT, paste0("node_", node), "store.at")

  d_full <- read_mat(file.path(store.at, "PC1max_wtd.csv")) -
            read_mat(file.path(store.at, "PC1min_wtd.csv"))
  m_full <- read_mat(file.path(store.at, MEAN_FILE))

  out_png_clean <- file.path(OUT_CLEAN, paste0("node_", node, "_", OUT_NAME_CLEAN))
  if (isTRUE(plot_node_panel_clean(d_full, m_full, out_png_clean))) {
    ok_n_clean <- ok_n_clean + 1
    cat("saved clean:", out_png_clean, "\n")
  }
}

cat("DONE. Clean panels saved:", ok_n_clean, "\n")


In [ ]:
# SPEC PROMEDIO PONDERADO DESDE WAVs ALIGNED


library(tuneR)

# ---- Parámetros ----
OUT_SPEC_DIR <- file.path(OUT_ROOT, "_SPECTRO_WTD_PNG")
dir.create(OUT_SPEC_DIR, recursive = TRUE, showWarnings = FALSE)

DB_FLOOR     <- -25
RIDGE_METHOD <- "peak"         # "peak" | "quantile"
RIDGE_Q      <- 0.90
RIDGE_SMOOTH <- 9              # impar; 0 sin suavizar

APPLY_FADE_WAVS <- TRUE
FADE_MS         <- 20

# SOLO VISUALIZACIÓN
FMAX_PLOT_KHZ <- 8

# GATE
ACTIVE_MARGIN_DB <- 2 

PNG_W   <- 2600
PNG_H   <- 1800
PNG_RES <- 300

# Halo de la cresta (switch)
RIDGE_USE_HALO <- TRUE   # <<< OFF por defecto (TRUE para contorno blanco)
RIDGE_HALO_COL <- "white"
RIDGE_HALO_LWD <- 4.5
RIDGE_COL      <- "black"
RIDGE_LWD      <- 2.5

# STFT (cálculos en 0–10 kHz)
N_FFT <- WL
HOP   <- as.integer(round(N_FFT * (1 - OVLP/100)))
WIN   <- 0.5 - 0.5*cos(2*pi*(0:(N_FFT-1))/(N_FFT-1))
COLS_GRAY <- gray.colors(256, start = 1, end = 0)

#  helpers 
read_wav_num <- function(path) {
  w <- tuneR::readWave(path)
  y <- as.numeric(w@left) / (2^(w@bit - 1))
  list(y = y, fs = w@samp.rate)
}

fade_in_out <- function(y, fs, fade_ms = 20) {
  n <- as.integer(round(fs * (fade_ms/1000)))
  if (n > 0 && 2*n < length(y)) {
    y[1:n] <- y[1:n] * seq(0, 1, length.out = n)
    y[(length(y)-n+1):length(y)] <- y[(length(y)-n+1):length(y)] * seq(1, 0, length.out = n)
  }
  y
}

stft_power <- function(y, n_fft, hop, win) {
  n <- length(y)
  n_frames <- as.integer(floor((n - n_fft) / hop) + 1)
  P <- matrix(0, nrow = n_fft/2 + 1, ncol = n_frames)
  for (j in 1:n_frames) {
    i0 <- (j - 1) * hop + 1
    frame <- y[i0:(i0 + n_fft - 1)] * win
    F <- fft(frame)[1:(n_fft/2 + 1)]
    P[, j] <- (Mod(F)^2)
  }
  P
}

# smooth dentro de bloques activos 
smooth_by_runs <- function(ridge, active, k = 9) {
  if (!k || k < 3 || (k %% 2 == 0)) return(ridge)
  r <- ridge
  rr <- rle(active)
  ends <- cumsum(rr$lengths)
  starts <- ends - rr$lengths + 1
  for (i in seq_along(rr$values)) {
    if (!rr$values[i]) next
    ii <- starts[i]:ends[i]
    seg <- r[ii]
    ok <- which(is.finite(seg))
    if (length(seg) >= k && length(ok) >= k) {
      seg[ok] <- stats::runmed(seg[ok], k = k, endrule = "median")
      r[ii] <- seg
    }
  }
  r
}

# dibujar cresta por segmentos
draw_ridge_segments <- function(x, ridge, active_plot,
                                use_halo = FALSE,
                                halo_col = "white", halo_lwd = 5.5,
                                line_col = "black", line_lwd = 2.5) {
  rr <- rle(active_plot)
  ends <- cumsum(rr$lengths)
  starts <- ends - rr$lengths + 1
  for (i in seq_along(rr$values)) {
    if (!rr$values[i]) next
    ii <- starts[i]:ends[i]
    if (isTRUE(use_halo)) {
      lines(x[ii], ridge[ii], col = halo_col, lwd = halo_lwd)
    }
    lines(x[ii], ridge[ii], col = line_col, lwd = line_lwd)
  }
}

#  loop nodos 
for (node in nodes) {

  ali.full <- file.path(OUT_ROOT, paste0("node_", node), "wav.at", "Aligned")
  wavs <- sort(list.files(ali.full, pattern="\\.wav$", full.names=TRUE))
  wavs_k <- wavs[1:min(K_USE, length(wavs))]

  d <- as.numeric(sub(".*_d([0-9.]+)_.*", "\\1", basename(wavs_k)))
  tau_node <- d[min(K_USE, length(d))]
  tau_use  <- if (TAU_MODE == "per_node") tau_node else TAU
  w <- kernel_w(d, tau_use)

  cat("\n=== Node", node, "| K_USE =", length(wavs_k),
      "| tau =", round(tau_use, 6),
      "| n_eff =", round(n_eff(w), 3), "===\n")

  ys <- lapply(wavs_k, read_wav_num)
  fs <- ys[[1]]$fs
  dur <- TLIM[2] - TLIM[1]
  N_target <- as.integer(round(dur * fs))

  ylist <- lapply(ys, function(z) {
    y <- z$y
    if (length(y) >= N_target) y <- y[1:N_target] else y <- c(y, rep(0, N_target - length(y)))
    if (APPLY_FADE_WAVS) y <- fade_in_out(y, fs, FADE_MS)
    y
  })

  #  promedio ponderado en potencia 
  P_acc <- NULL
  for (i in seq_along(ylist)) {
    P_i <- stft_power(ylist[[i]], N_FFT, HOP, WIN)
    if (is.null(P_acc)) P_acc <- 0 * P_i
    P_acc <- P_acc + w[i] * P_i
  }
  P_avg <- P_acc / sum(w)

  freqs_hz <- (0:(N_FFT/2)) * fs / N_FFT
  times_s  <- (0:(ncol(P_avg)-1)) * HOP / fs

  # recorte de cálculo hasta FLIM[2] (10 kHz)
  fmax_hz <- FLIM[2] * 1000
  kmax <- max(which(freqs_hz <= fmax_hz))
  P_avg <- P_avg[1:kmax, , drop = FALSE]
  freqs_hz <- freqs_hz[1:kmax]

  #  regrillar a (Y_LENGTH x X_LENGTH) en 0–10 kHz 
  t_mesh <- seq(TLIM[1], TLIM[2], length.out = X_LENGTH)
  f_mesh_hz <- seq(FLIM[1]*1000, FLIM[2]*1000, length.out = Y_LENGTH)

  P_t <- matrix(0, nrow = nrow(P_avg), ncol = X_LENGTH)
  for (r in 1:nrow(P_avg)) {
    P_t[r, ] <- approx(times_s, P_avg[r, ], xout = t_mesh, rule = 2)$y
  }

  P_grid <- matrix(0, nrow = Y_LENGTH, ncol = X_LENGTH)
  for (j in 1:X_LENGTH) {
    P_grid[, j] <- approx(freqs_hz, P_t[, j], xout = f_mesh_hz, rule = 2)$y
  }

  #  dB relativo global (max global = 0 dB) 
  eps <- 1e-12
  S_db_global <- 10 * log10(P_grid + eps)
  S_db_global <- S_db_global - max(S_db_global, na.rm = TRUE)

  #  GATE: frames activos vs silencios 
  col_max_global <- apply(S_db_global, 2, max, na.rm = TRUE)
  active <- col_max_global > (DB_FLOOR + ACTIVE_MARGIN_DB)

  #  frame-norm SOLO si frame activo; si no, queda blanco 
  S_db_plot <- S_db_global
  for (j in 1:ncol(S_db_plot)) {
    if (!active[j]) S_db_plot[, j] <- DB_FLOOR
    else            S_db_plot[, j] <- S_db_global[, j] - col_max_global[j]
  }
  S_db_plot[S_db_plot < DB_FLOOR] <- DB_FLOOR

  #  cresta (raw) SOLO en frames activos 
  f_khz_full <- f_mesh_hz / 1000
  ridge_raw <- rep(NA_real_, length(t_mesh))

  if (RIDGE_METHOD == "peak") {
    for (j in 1:length(t_mesh)) {
      if (!active[j]) next
      ridge_raw[j] <- f_khz_full[which.max(S_db_plot[, j])]
    }
  } else {
    P_rel <- 10^(S_db_plot / 10)
    for (j in 1:length(t_mesh)) {
      if (!active[j]) next
      v <- P_rel[, j]; v[!is.finite(v)] <- 0; v <- pmax(v, 0)
      s <- sum(v); if (!is.finite(s) || s <= 0) next
      cs <- cumsum(v) / s
      idx <- which(cs >= RIDGE_Q)[1]
      if (!is.na(idx)) ridge_raw[j] <- f_khz_full[idx]
    }
  }

  #  smooth solo dentro de cada bloque activo 
  ridge <- smooth_by_runs(ridge_raw, active, k = RIDGE_SMOOTH)

  # PLOT 0–8 kHz
  keep <- which(f_khz_full <= FMAX_PLOT_KHZ)
  S_plot_cut <- S_db_plot[keep, , drop = FALSE]
  f_khz_cut  <- f_khz_full[keep]

  #  PNG 
  node_tag <- if (grepl("^[0-9]+$", node)) sprintf("%03d", as.integer(node)) else node
  out_png <- file.path(OUT_SPEC_DIR, paste0("spec_mean_wtd_node_", node_tag, ".png"))

  png(out_png, width = PNG_W, height = PNG_H, res = PNG_RES)
  layout(matrix(c(1,2), 1, 2), widths = c(4.6, 1.2))

  par(mar = c(5, 5, 1.5, 1) + 0.1, xaxs = "i", yaxs = "i")
  plot.new()
  plot.window(xlim = c(TLIM[1], TLIM[2]), ylim = c(0, FMAX_PLOT_KHZ), xaxs = "i", yaxs = "i")

  image(t_mesh, f_khz_cut, t(S_plot_cut),
        col = COLS_GRAY, zlim = c(DB_FLOOR, 0),
        xlab = "Time (s)", ylab = "Frequency (kHz)",
        axes = FALSE, add = TRUE)

  box()
  axis(1, at = TLIM[1]:TLIM[2], labels = TLIM[1]:TLIM[2])
  axis(2, at = 0:FMAX_PLOT_KHZ, labels = 0:FMAX_PLOT_KHZ, las = 1)

  # dibuja la cresta por segmentos
  active_plot <- active & is.finite(ridge) & (ridge <= FMAX_PLOT_KHZ)
  if (any(active_plot)) {
    draw_ridge_segments(
      t_mesh, ridge, active_plot,
      use_halo = RIDGE_USE_HALO,
      halo_col = RIDGE_HALO_COL, halo_lwd = RIDGE_HALO_LWD,
      line_col = RIDGE_COL,      line_lwd = RIDGE_LWD
    )
  }

  # colorbar
  par(mar = c(5, 1, 1.5, 4) + 0.1)
  zseq <- seq(DB_FLOOR, 0, length.out = 256)
  image(x = 1, y = zseq, z = matrix(zseq, nrow = 1),
        col = COLS_GRAY, xlab = "", ylab = "", axes = FALSE)
  axis(4)
  mtext("dB (rel.; frame-norm for plot)", side = 4, line = 2.4)

  dev.off()

  cat("saved:", out_png, "\n")
}

cat("DONE. All PNGs in:", OUT_SPEC_DIR, "\n")


In [ ]:
# SPEC PROMEDIO PONDERADO DESDE WAVs ALIGNED

library(tuneR)

#  Parámetros 
OUT_SPEC_DIR_CLEAN <- file.path(OUT_ROOT, "_SPECTRO_WTD_PNG_CLEAN")
dir.create(OUT_SPEC_DIR_CLEAN, recursive = TRUE, showWarnings = FALSE)

DB_FLOOR     <- -25
RIDGE_METHOD <- "peak"         # "peak" | "quantile"
RIDGE_Q      <- 0.90
RIDGE_SMOOTH <- 9              # impar; 0 sin suavizar

APPLY_FADE_WAVS <- TRUE
FADE_MS         <- 20

# SOLO VISUALIZACIÓN
FMAX_PLOT_KHZ <- 8

#  GATE
ACTIVE_MARGIN_DB <- 2

# PNG alta resolución
PNG_W   <- 2600
PNG_H   <- 1800
PNG_RES <- 300

#  Halo de la cresta (switch) 
RIDGE_USE_HALO <- TRUE   #  OFF por defecto
RIDGE_HALO_COL <- "white"
RIDGE_HALO_LWD <- 4.5
RIDGE_COL      <- "black"
RIDGE_LWD      <- 2.5

# STFT (cálculos en 0–10 kHz)
N_FFT <- WL
HOP   <- as.integer(round(N_FFT * (1 - OVLP/100)))
WIN   <- 0.5 - 0.5*cos(2*pi*(0:(N_FFT-1))/(N_FFT-1))
COLS_GRAY <- gray.colors(256, start = 1, end = 0)

#  helpers 
read_wav_num <- function(path) {
  w <- tuneR::readWave(path)
  y <- as.numeric(w@left) / (2^(w@bit - 1))
  list(y = y, fs = w@samp.rate)
}

fade_in_out <- function(y, fs, fade_ms = 20) {
  n <- as.integer(round(fs * (fade_ms/1000)))
  if (n > 0 && 2*n < length(y)) {
    y[1:n] <- y[1:n] * seq(0, 1, length.out = n)
    y[(length(y)-n+1):length(y)] <- y[(length(y)-n+1):length(y)] * seq(1, 0, length.out = n)
  }
  y
}

stft_power <- function(y, n_fft, hop, win) {
  n <- length(y)
  n_frames <- as.integer(floor((n - n_fft) / hop) + 1)
  P <- matrix(0, nrow = n_fft/2 + 1, ncol = n_frames)
  for (j in 1:n_frames) {
    i0 <- (j - 1) * hop + 1
    frame <- y[i0:(i0 + n_fft - 1)] * win
    F <- fft(frame)[1:(n_fft/2 + 1)]
    P[, j] <- (Mod(F)^2)
  }
  P
}

smooth_by_runs <- function(ridge, active, k = 9) {
  if (!k || k < 3 || (k %% 2 == 0)) return(ridge)
  r <- ridge
  rr <- rle(active)
  ends <- cumsum(rr$lengths)
  starts <- ends - rr$lengths + 1
  for (i in seq_along(rr$values)) {
    if (!rr$values[i]) next
    ii <- starts[i]:ends[i]
    seg <- r[ii]
    ok <- which(is.finite(seg))
    if (length(seg) >= k && length(ok) >= k) {
      seg[ok] <- stats::runmed(seg[ok], k = k, endrule = "median")
      r[ii] <- seg
    }
  }
  r
}

# dibujar cresta por segmentos
draw_ridge_segments <- function(x, ridge, active_plot,
                                use_halo = FALSE,
                                halo_col = "white", halo_lwd = 5.5,
                                line_col = "black", line_lwd = 2.5) {
  rr <- rle(active_plot)
  ends <- cumsum(rr$lengths)
  starts <- ends - rr$lengths + 1
  for (i in seq_along(rr$values)) {
    if (!rr$values[i]) next
    ii <- starts[i]:ends[i]
    if (isTRUE(use_halo)) lines(x[ii], ridge[ii], col = halo_col, lwd = halo_lwd)
    lines(x[ii], ridge[ii], col = line_col, lwd = line_lwd)
  }
}

#  loop nodos 
for (node in nodes) {

  ali.full <- file.path(OUT_ROOT, paste0("node_", node), "wav.at", "Aligned")
  wavs <- sort(list.files(ali.full, pattern="\\.wav$", full.names=TRUE))
  wavs_k <- wavs[1:min(K_USE, length(wavs))]

  d <- as.numeric(sub(".*_d([0-9.]+)_.*", "\\1", basename(wavs_k)))
  tau_node <- d[min(K_USE, length(d))]
  tau_use  <- if (TAU_MODE == "per_node") tau_node else TAU
  w <- kernel_w(d, tau_use)

  cat("\n=== Node", node, "| K_USE =", length(wavs_k),
      "| tau =", round(tau_use, 6),
      "| n_eff =", round(n_eff(w), 3), "===\n")

  ys <- lapply(wavs_k, read_wav_num)
  fs <- ys[[1]]$fs
  dur <- TLIM[2] - TLIM[1]
  N_target <- as.integer(round(dur * fs))

  ylist <- lapply(ys, function(z) {
    y <- z$y
    if (length(y) >= N_target) y <- y[1:N_target] else y <- c(y, rep(0, N_target - length(y)))
    if (APPLY_FADE_WAVS) y <- fade_in_out(y, fs, FADE_MS)
    y
  })

  #  promedio ponderado en potencia 
  P_acc <- NULL
  for (i in seq_along(ylist)) {
    P_i <- stft_power(ylist[[i]], N_FFT, HOP, WIN)
    if (is.null(P_acc)) P_acc <- 0 * P_i
    P_acc <- P_acc + w[i] * P_i
  }
  P_avg <- P_acc / sum(w)

  freqs_hz <- (0:(N_FFT/2)) * fs / N_FFT
  times_s  <- (0:(ncol(P_avg)-1)) * HOP / fs

  # recorte de cálculo hasta FLIM[2] (10 kHz)
  fmax_hz <- FLIM[2] * 1000
  kmax <- max(which(freqs_hz <= fmax_hz))
  P_avg <- P_avg[1:kmax, , drop = FALSE]
  freqs_hz <- freqs_hz[1:kmax]

  #  regrillar a (Y_LENGTH x X_LENGTH) en 0–10 kHz 
  t_mesh <- seq(TLIM[1], TLIM[2], length.out = X_LENGTH)
  f_mesh_hz <- seq(FLIM[1]*1000, FLIM[2]*1000, length.out = Y_LENGTH)

  P_t <- matrix(0, nrow = nrow(P_avg), ncol = X_LENGTH)
  for (r in 1:nrow(P_avg)) {
    P_t[r, ] <- approx(times_s, P_avg[r, ], xout = t_mesh, rule = 2)$y
  }

  P_grid <- matrix(0, nrow = Y_LENGTH, ncol = X_LENGTH)
  for (j in 1:X_LENGTH) {
    P_grid[, j] <- approx(freqs_hz, P_t[, j], xout = f_mesh_hz, rule = 2)$y
  }

  #  dB relativo global (max global = 0 dB) 
  eps <- 1e-12
  S_db_global <- 10 * log10(P_grid + eps)
  S_db_global <- S_db_global - max(S_db_global, na.rm = TRUE)

  #  GATE: frames activos vs silencios 
  col_max_global <- apply(S_db_global, 2, max, na.rm = TRUE)
  active <- col_max_global > (DB_FLOOR + ACTIVE_MARGIN_DB)

  #  frame-norm SOLO si frame activo; si no, lo dejamos floor 
  S_db_plot <- S_db_global
  for (j in 1:ncol(S_db_plot)) {
    if (!active[j]) S_db_plot[, j] <- DB_FLOOR
    else            S_db_plot[, j] <- S_db_global[, j] - col_max_global[j]
  }
  S_db_plot[S_db_plot < DB_FLOOR] <- DB_FLOOR

  #  cresta (raw) SOLO en frames activos 
  f_khz_full <- f_mesh_hz / 1000
  ridge_raw <- rep(NA_real_, length(t_mesh))

  if (RIDGE_METHOD == "peak") {
    for (j in 1:length(t_mesh)) {
      if (!active[j]) next
      ridge_raw[j] <- f_khz_full[which.max(S_db_plot[, j])]
    }
  } else {
    P_rel <- 10^(S_db_plot / 10)
    for (j in 1:length(t_mesh)) {
      if (!active[j]) next
      v <- P_rel[, j]; v[!is.finite(v)] <- 0; v <- pmax(v, 0)
      s <- sum(v); if (!is.finite(s) || s <= 0) next
      cs <- cumsum(v) / s
      idx <- which(cs >= RIDGE_Q)[1]
      if (!is.na(idx)) ridge_raw[j] <- f_khz_full[idx]
    }
  }

  #  smooth solo dentro de cada bloque activo 
  ridge <- smooth_by_runs(ridge_raw, active, k = RIDGE_SMOOTH)

  #  SOLO PLOT: recorte a 0–8 kHz 
  keep <- which(f_khz_full <= FMAX_PLOT_KHZ)
  S_plot_cut <- S_db_plot[keep, , drop = FALSE]
  f_khz_cut  <- f_khz_full[keep]

  #  PNG LIMPIO (solo figura) 
  node_tag <- if (grepl("^[0-9]+$", node)) sprintf("%03d", as.integer(node)) else node
  out_png <- file.path(OUT_SPEC_DIR_CLEAN, paste0("spec_mean_wtd_node_", node_tag, "_clean.png"))

  png(out_png, width = PNG_W, height = PNG_H, res = PNG_RES)

  # ocupa TODO el device, sin márgenes
  par(mar = c(0, 0, 0, 0), oma = c(0, 0, 0, 0), xaxs = "i", yaxs = "i")
  par(fig = c(0, 1, 0, 1), new = FALSE)

  plot.new()
  plot.window(xlim = c(TLIM[1], TLIM[2]), ylim = c(0, FMAX_PLOT_KHZ), xaxs = "i", yaxs = "i")

  image(t_mesh, f_khz_cut, t(S_plot_cut),
        col = COLS_GRAY, zlim = c(DB_FLOOR, 0),
        axes = FALSE, xlab = "", ylab = "", add = TRUE)

  # cresta (sin ejes, sin caja)
  active_plot <- active & is.finite(ridge) & (ridge <= FMAX_PLOT_KHZ)
  if (any(active_plot)) {
    draw_ridge_segments(
      t_mesh, ridge, active_plot,
      use_halo = RIDGE_USE_HALO,
      halo_col = RIDGE_HALO_COL, halo_lwd = RIDGE_HALO_LWD,
      line_col = RIDGE_COL,      line_lwd = RIDGE_LWD
    )
  }

  dev.off()
  cat("saved clean:", out_png, "\n")
}

cat("DONE. Clean PNGs in:", OUT_SPEC_DIR_CLEAN, "\n")
